In [1]:
from pymongo import MongoClient
import pandas as pd
import folium
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster
import json
import analysis as f
import cleaning as cl
import geoquery as g

# Analysis

#### Lanzamiento de solicitudes mediante API así como importación de ficheros csv para obtener ubicaciones de los parámetros a tener en cuenta para la elección de nuestra ubicación. Esta información se va plasmando mediante capas en un mapa para obtener un resultado más visual.

- Importamos de nuevo la colección

In [2]:
client = MongoClient("mongodb://localhost/companies")

In [3]:
db = client.get_database()

In [4]:
query = {"$and":[{'city':'San Francisco','tech_company':'yes'}]}

In [5]:
data = list(db["cleaned_companies"].find(query,{"_id":0}))

tech_companies = pd.DataFrame(data)

In [6]:
tech_companies.to_json("../OUTPUT/tech_companies", orient="records")

- Creamos mapa con las coordenadas de San Francisco

In [7]:
start_lat = 37.773972
start_lon = -122.431297
heat_m = folium.Map(location=[start_lat, start_lon],tiles='cartodbpositron', zoom_start=15)

- Incluimos un mapa de calor con todas las compañías tecnológicas de San Francisco

In [8]:
sanF_group = folium.FeatureGroup(name='Sf_companies')
HeatMap(data=tech_companies[['latitude', 'longitude']]).add_to(sanF_group)
sanF_group.add_to(heat_m)

heat_m

- Obtenemos las coordenadas del aeropuerto de San Francisco

In [9]:
airport = f.googleApi('San Francisco Airport')

- Marcamos el punto en el mapa

In [10]:
uni_lon = airport['results'][0]['geometry']['location']['lng']
uni_lat = airport['results'][0]['geometry']['location']['lat']

Marker([uni_lat, uni_lon], icon=folium.Icon(color='blue', icon='plane', prefix='fa')).add_to(heat_m)
heat_m

- Importamos el csv que contiene todos los starbucks

In [12]:
starbucks = pd.read_csv('../INPUT/starbucks.csv')

- Filtramos por los starbucks que hay en San Francisco

In [13]:
filt = starbucks['City'] == 'San Francisco'
starbucks = starbucks[filt]

- Adaptamos las coordenadas

In [14]:
starbucks["location"] = starbucks[["Latitude","Longitude"]].apply(lambda x:cl.coorFormat(x.Latitude,x.Longitude), axis=1)

- Guardamos como json para incluir colección en MongoDB

In [ ]:
starbucks.to_json("../OUTPUT/starbucks", orient="records")

- Marcamos en el mapa todos los starbucks de San Francisco con su color representativo, el verde

In [15]:
for i in range(len(starbucks.Latitude)):
    Circle(
        location=[starbucks.iloc[i]['Latitude'],starbucks.iloc[i]['Longitude']],
        radius=15,
        color='green',
        fill=True,
        fill_color='green').add_to(heat_m)

In [16]:
heat_m

- Obtenemos las coordenadas de los pubs de San Francisco (Google solo me proporciona los primeros 20)

In [17]:
pubs = f.googlePlaces('pubs in san francisco')

- Guardamos como json para incluir colección en MongoDB

In [ ]:
with open('pubs.json', 'w') as fp:
    json.dump(pubs, fp)

- Sacamos las coordenadas de los pubs

In [18]:
pubs2 = []

for i in range(len(pubs['results'])):
    pubs2.append(pubs['results'][i]['geometry']['location'])

- Convertimos en DataFrame para adaptar las coordenadas

In [19]:
df_pubs = pd.DataFrame(pubs2)

In [20]:
df_pubs["location"] = df_pubs[["lat","lng"]].apply(lambda x:cl.coorFormat(x.lat,x.lng), axis=1)

- Guardamos como csv como copia de seguridad

In [ ]:
df_pubs.to_csv("../OUTPUT/pubs.csv")

- Marcamos en el mapa los pubs obtenidos, en color rojo

In [21]:
for i in range(len(df_pubs.lat)):
    Circle(
        location=[df_pubs.iloc[i]['lat'],df_pubs.iloc[i]['lng']],
        radius=15,
        color='red',
        fill=True,
        fill_color='red').add_to(heat_m)

In [22]:
heat_m

-  Obtenemos las coordenadas de los colegios en San Francisco (Google solo me proporciona los primeros 20)

In [23]:
schools = f.googlePlaces('schools in san francisco')

- Guardamos como json para incluir colección en MongoDB

In [ ]:
with open('schools.json', 'w') as fp:
    json.dump(pubs, fp)

- Sacamos las coordenadas

In [24]:
schools2 = []

for i in range(len(schools['results'])):
    schools2.append(schools['results'][i]['geometry']['location'])

- Convertimos en DataFrame para adaptar las coordenadas

In [25]:
df_schools = pd.DataFrame(schools2)

In [26]:
df_schools["location"] = df_schools[["lat","lng"]].apply(lambda x:cl.coorFormat(x.lat,x.lng), axis=1)

- Guardamos como csv como copia de seguridad

In [ ]:
df_schools.to_csv("../OUTPUT/schools.csv")

- Marcamos en el mapa los colegios obtenidos, en color amarillo

In [27]:
for i in range(len(df_schools.lat)):
    Circle(
        location=[df_schools.iloc[i]['lat'],df_schools.iloc[i]['lng']],
        radius=15,
        color='yellow',
        fill=True,
        fill_color='yellow').add_to(heat_m)

In [28]:
heat_m

# Final Decision

#### Teniendo todas las coordenadas de los lugares de nuestro interés, realizamos geoqueries a MongoDB para obtener las compañías tecnológicas que cumplen con todas estas características (estar posicionadas como máximo a 2km de los lugares de interés). La compañía que más coincidencias reciba, será la elegida para posicionar nuestras oficinas en el mismo edificio.

In [30]:
tech_comp = tech_companies[['name','category_code','number_of_employees','founded_year','latitude','longitude','location']]

- Creamos una lista con las localizaciones de las compañías para poder compararlas con los demás parámetros

In [31]:
list_techcomp = []

for i in range(len(tech_comp)):
    list_techcomp.append(tech_comp['location'][i])

- Generamos las geoquerys para cada compañía tecnológica de San Francisco en cada parámetros para obtener cuántos de éstos se sitúan a 800m como máximo

In [33]:
starbucks_matches = g.geoquery(list_techcomp,'starbucks')
pubs_matches = g.geoquery(list_techcomp,'pubs')
schools_matches = g.geoquery(list_techcomp,'schools')

- Incluimos los matches en el DataFrame de las compañías

In [34]:
tech_comp['starbucks800m'] = starbucks_matches
tech_comp['pubs800m'] = pubs_matches
tech_comp['schools800m'] = schools_matches

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

In [35]:
tech_comp.head()

,name,category_code,number_of_employees,founded_year,latitude,longitude,location,starbucks800m,pubs800m,schools800m
0,StumbleUpon,web,NaN,2002.0,37.775196,-122.419204,"{'type': 'Point', 'coordinates': [-122.419204,...",30,7,2
1,Scribd,news,50.0,2007.0,37.789634,-122.404052,"{'type': 'Point', 'coordinates': [-122.404052,...",50,7,0
2,Technorati,advertising,35.0,2002.0,37.779558,-122.393041,"{'type': 'Point', 'coordinates': [-122.393041,...",45,5,0
3,Kyte,games_video,40.0,2006.0,37.788482,-122.409173,"{'type': 'Point', 'coordinates': [-122.409173,...",51,8,1
4,Prosper,finance,NaN,2006.0,37.789760,-122.402524,"{'type': 'Point', 'coordinates': [-122.402524,...",50,7,0


- Creamos una columna con el sumatorio de las coincidencias y según su grado de importancia

In [36]:
tech_comp['matches'] = tech_comp.starbucks800m*0.5 + tech_comp.pubs800m*0.35 + tech_comp.schools800m*0.15

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


- Ordenamos de mayor a menor

In [38]:
tech_comp.sort_values(by=['matches'], ascending = False).head()

,name,category_code,number_of_employees,founded_year,latitude,longitude,location,starbucks800m,pubs800m,schools800m,matches
162,GIS Planning,ecommerce,16.0,2007.0,37.784137,-122.408646,"{'type': 'Point', 'coordinates': [-122.408646,...",55,7,1,30.10
311,EVault,software,NaN,1997.0,37.785050,-122.399817,"{'type': 'Point', 'coordinates': [-122.399817,...",54,7,0,29.45
222,ON24,web,NaN,1998.0,37.785033,-122.400474,"{'type': 'Point', 'coordinates': [-122.4004738...",54,7,0,29.45
85,Mobissimo,search,15.0,2003.0,37.786240,-122.398970,"{'type': 'Point', 'coordinates': [-122.39897, ...",53,7,0,28.95
31,eBuddy,messaging,90.0,2003.0,37.787683,-122.410943,"{'type': 'Point', 'coordinates': [-122.410943,...",52,8,1,28.95


### Con esto, obtenemos la empresa GIS Planning como la mejor ubicada, ya que satisface la mayoría de nuestras preferencias. Por lo que, la decisión final es situar nuestras oficinas en el mismo edificio donde se encuentran

In [39]:
company = tech_comp.name == 'GIS Planning'
select = tech_comp[company]
final_decisionLat = select['latitude']
final_decisionLon = select['longitude']

In [43]:
print(final_decisionLat)
print(final_decisionLon)

162    37.784137
Name: latitude, dtype: float64
162   -122.408646
Name: longitude, dtype: float64


In [41]:
Marker([final_decisionLat, final_decisionLon], icon=folium.Icon(color='black', icon='briefcase', prefix='fa')).add_to(heat_m)
heat_m

In [42]:
heat_m.save('../OUTPUT/SanFrancisco_office')